In [ ]:
from __future__ import print_function

"""
A script to run cqed_rhf method on the formaldehyde coupled to a photon with 
different polarizations and coupling magnitude |\lambda| = 0.2 atomic units
to produce data from Table I of McTague and Foley 
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
from helper_cqed_rhf import *
from helper_ciss_prism import *
from matplotlib import pyplot as plt

# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
psi4.core.set_output_file('output.dat', False)

numpy_memory = 2

In [ ]:
# rhf/cc-pVDZ optimized geometry of formaldehyde
molstr = """

0 1
O 0.0000000000 0.0000000000 5.91268220e-01
C 0.0000000000 0.0000000000 -5.90400099e-01
H 0.0000000000 9.32184336e-01 -1.17703144e+00
H 0.0000000000 -9.32184336e-01 -1.17703144e+00
no_reorient
symmetry c1
"""

# options dict
options_dict = {'basis': 'cc-pVDZ',
               'save_jk': True, 
               'scf_type': 'pk',
               'e_convergence' : 1e-8,
               'd_convergence' : 1e-7}



In [ ]:
# set cavity parameters
om = 0.3780197
lam = np.array([0, 0, 0.00])

# cqed-rhf
#cqed_rhf_dict = cqed_rhf(lam, molstr, options_dict)

# cqed-cis
cqed_cis_dict = cs_cqed_cis(lam, om, molstr, options_dict)

In [ ]:
scf_e = cqed_cis_dict["RHF ENERGY"]
cqed_scf_e = cqed_cis_dict["CQED-RHF ENERGY"]

ciss_pf_e = cqed_cis_dict["CISS-PF ENERGY"]
ciss_jc_e = cqed_cis_dict["CISS-JC ENERGY"]
cis_pf_e = cqed_cis_dict["CIS-PF ENERGY"]
cis_jc_e = cqed_cis_dict["CIS-JC ENERGY"]
cis_dse_e = cqed_cis_dict["CIS-DSE ENERGY"]


print(ciss_pf_e[1:8])
print(ciss_jc_e[1:8])
print(cis_pf_e[0:7])
print(cis_jc_e[0:7])
print(cis_dse_e[0:7])


In [ ]:
lam_z = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]

In [ ]:

ciss_pf = np.zeros((5,len(lam_z)))
ciss_jc = np.zeros((5,len(lam_z)))
cis_pf = np.zeros((5,len(lam_z)))
cis_jc = np.zeros((5,len(lam_z)))

# lambda vector
#lam = np.array([0,0,lam_z])

# loop over the different bond-lengths, create different instances
# of MgH+ molecule and compute and store various quantities
ctr = 0
for l in lam_z:
    lam = np.array([0, 0, l])
    
    cqed_cis_dict = cs_cqed_cis(lam, om, molstr, options_dict)
    # get cqed_scf_e
    cqed_scf_e = cqed_cis_dict["CQED-RHF ENERGY"]
    
    # get eigenvalues from CISS-PF
    ciss_pf_e = cqed_cis_dict["CISS-PF ENERGY"]
    # get eigenvalues from CISS-JC
    ciss_jc_e = cqed_cis_dict["CISS-JC ENERGY"]
    # get eigenvalues from CIS-PF
    cis_pf_e = cqed_cis_dict["CIS-PF ENERGY"]
    # get eigenvalues from CIS-JC
    cis_jc_e = cqed_cis_dict["CIS-JC ENERGY"]
    

    
    ciss_pf[:5,ctr] = cqed_scf_e+ciss_pf_e[:5]
    ciss_jc[:5,ctr] = cqed_scf_e+ciss_jc_e[:5]
    
    cis_pf[0,ctr] = cqed_scf_e
    cis_jc[0,ctr] = cqed_scf_e

    cis_pf[1:5,ctr] = cqed_scf_e + cis_pf_e[0:4]
    cis_jc[1:5,ctr] = cqed_scf_e + cis_jc_e[0:4]
    ctr += 1
    
    

In [ ]:
from matplotlib import pyplot as plt

plt.plot(lam_z, ciss_pf[2,:], 'r-o', label='CISS-PF')
plt.plot(lam_z, ciss_jc[2,:], 'g-o', label='CISS-JC')
plt.plot(lam_z, cis_pf[2,:], 'b-o', label='CIS-PF')
plt.plot(lam_z, cis_jc[2,:], 'm-o', label='CIS-JC')
plt.plot(lam_z, ciss_pf[3,:], 'r-o')
plt.plot(lam_z, ciss_jc[3,:], 'g-o')
plt.plot(lam_z, cis_pf[3,:], 'b-o')
plt.plot(lam_z, cis_jc[3,:], 'm-o')
plt.plot(lam_z, ciss_pf[0,:], 'r--')
plt.plot(lam_z, ciss_jc[0,:], 'g--')
plt.plot(lam_z, cis_pf[0,:], 'b--')
plt.plot(lam_z, cis_jc[0,:], 'm--')

plt.xlabel("$\lambda$ (a.u.)")
plt.ylabel("Energy (a.u.)")
plt.legend()
#plt.savefig("Compare_Prism.png")
plt.show()

In [ ]:
plt.plot(lam_z, ciss_pf[0,:], 'r-o', label='CISS-PF')
plt.plot(lam_z, ciss_jc[0,:], 'g-o', label='CISS-JC')
plt.plot(lam_z, cis_pf[0,:], 'b-o', label='CIS-PF')
plt.plot(lam_z, cis_jc[0,:], 'm-o', label='CIS-JC')

plt.xlabel("$\lambda$ (a.u.)")
plt.ylabel("Energy (a.u.)")
plt.legend()
#plt.savefig("Compare_Prism.png")
plt.show()